In [ ]:
import os
os.chdir('..')

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import torch
import torchvision
from pytorch_lightning.callbacks import ModelCheckpoint
from src.model.lit_module import LitModule
from src.data.dataset import VideoLabelDataset
import src.constants as const
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from torch.utils.data import DataLoader
from src.data.dataset import (VideoLabelDataset,
                              VideoFolderPathToTensor,
                              VideoResize)
import plotly
import numpy as np
import pandas as pd
import yaml
import os
%load_ext autoreload
%autoreload 2

In [ ]:
dataset = VideoLabelDataset(
            const.LABELS_TABLE_QA_PATH,
            img_transform=torchvision.transforms.Compose([
                VideoFolderPathToTensor(),
                VideoResize(const.IMG_SIZE)]))

In [ ]:
dataloader = DataLoader(dataset, batch_size=100, num_workers=6)

In [ ]:
MODEL_VERSION = 31
hparams_path = f'./lightning_logs/version_{MODEL_VERSION}/hparams.yaml'
with open(hparams_path, 'r') as stream:
        hparam_dct = yaml.safe_load(stream)
ckpt_file_name = os.listdir(f'./lightning_logs/version_{MODEL_VERSION}/checkpoints/')[-1]
ckpt_file_path = f'./lightning_logs/version_{MODEL_VERSION}/checkpoints/{ckpt_file_name}'
model = LitModule.load_from_checkpoint(ckpt_file_path)

In [ ]:
videos, answers, hidden_states, vid_folder  = iter(dataloader).next()

In [ ]:
hidden_states_pred = model.eval()(videos)

In [ ]:
df_hidden_states_pred = pd.DataFrame(hidden_states_pred.detach().numpy(), columns=[f'lat_neuron_{c}' for c in range(4)])
df_hidden_states = pd.DataFrame(hidden_states.detach().numpy(), columns=const.HIDDEN_STATE_COLS)

In [ ]:
df_hidden_states

In [ ]:
df_hidden_states_pred

In [ ]:
fig = make_subplots(rows=1, cols=4)
i=1
for c1, c2 in zip(const.HIDDEN_STATE_COLS, ['lat_neuron_0', 'lat_neuron_1', 'lat_neuron_2', 'lat_neuron_3']):
    fig.add_trace(go.Scatter(x=df_hidden_states[c1], y=df_hidden_states_pred[c2],
                        mode='markers',
                        marker_color='#1f77b4',
                        name=f'{c1} over {c2}'), row=1, col=0+i)
    i+=1
fig.show()

In [ ]:
hidden_states_pred = model.eval()(videos)
answers_pred = model._shared_eval(videos, questions)

In [ ]:
df_answers_pred = pd.DataFrame(answers_pred.detach().numpy(), columns=[f'{c}_pred' for c in const.ANSWER_COLS])
df_hidden_states_pred = pd.DataFrame(hidden_states_pred.detach().numpy(), columns=[f'lat_neuron_{c}' for c in range(4)])
df_answers = pd.DataFrame(answers.detach().numpy(), columns=const.ANSWER_COLS)
df_hidden_states = pd.DataFrame(hidden_states.detach().numpy(), columns=const.HIDDEN_STATE_COLS)
df_questions = pd.DataFrame(questions.detach().numpy(), columns=[const.QUESTION_COL])

In [ ]:
fig = make_subplots(rows=1, cols=4)
for i, c in enumerate(const.ANSWER_COLS):
    fig.add_trace(go.Scatter(x=df_answers[c], y=df_answers_pred[f'{c}_pred'],
                        mode='markers',
                        marker_color='#1f77b4',
                        name=c), row=1, col=i+1)
    fig.update_xaxes(title_text=c, row=1, col=i+1)
fig.update_layout(title_text="Predicted answer over optimal answer", width=1500, showlegend=False)
fig.update_yaxes(title_text="Predicted answers", col=1)
fig.show()

In [ ]:
fig = make_subplots(rows=1, cols=4)
for i, c in enumerate(const.ANSWER_COLS):
    fig.add_trace(go.Scatter(x=df_answers[c], y=df_answers_pred[f'{c}_pred'],
                        mode='markers',
                        marker_color='#1f77b4',
                        name=c), row=1, col=i+1)
    fig.update_xaxes(title_text=c, row=1, col=i+1)
fig.update_layout(title_text="Predicted answer over optimal answer", width=1500, showlegend=False)
fig.update_yaxes(title_text="Predicted answers", col=1)
fig.show()

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=4, cols=4)

for i, hs in enumerate(df_hidden_states.columns):
    for j, hs_pred in enumerate(df_hidden_states_pred.columns):
        fig.add_trace(go.Scatter(x=df_hidden_states[hs], y=df_hidden_states_pred[hs_pred],
                            mode='markers', name=f'activation {hs_pred} over box_x',
                                marker_color='#1f77b4'),
                     row=i+1, col=j+1)

# # Update xaxis properties
for i in range(hparam_dct['enc_dim_lat_space']):
    fig.update_xaxes(title_text=const.HIDDEN_STATE_COLS[0], row=1, col=i+1)
    fig.update_xaxes(title_text=const.HIDDEN_STATE_COLS[1], row=2, col=i+1)
    fig.update_xaxes(title_text=const.HIDDEN_STATE_COLS[2], row=3, col=i+1)
#     fig.update_xaxes(title_text=const.HIDDEN_STATE_COLS[3], row=4, col=i+1)

for l in range(len(const.HIDDEN_STATE_COLS)):
        fig.update_yaxes(title_text=f"Activation l_{l}", row=l+1, col=1)

fig.update_layout(height=1000, width=1200, title_text="Latent neuron activations vs. hidden states", showlegend=False)
fig.show()

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=4, cols=4)

for i, hs in enumerate(df_hidden_states.columns):
    for j, hs_pred in enumerate(df_hidden_states_pred.columns):
        fig.add_trace(go.Scatter(x=df_hidden_states[hs], y=df_hidden_states_pred[hs_pred],
                            mode='markers', name=f'activation {hs_pred} over box_x',
                                marker_color='#1f77b4'),
                     row=i+1, col=j+1)

# # Update xaxis properties
for i in range(hparam_dct['enc_dim_lat_space']):
    fig.update_xaxes(title_text=const.HIDDEN_STATE_COLS[0], row=1, col=i+1)
    fig.update_xaxes(title_text=const.HIDDEN_STATE_COLS[1], row=2, col=i+1)
    fig.update_xaxes(title_text=const.HIDDEN_STATE_COLS[2], row=3, col=i+1)
#     fig.update_xaxes(title_text=const.HIDDEN_STATE_COLS[3], row=4, col=i+1)

for l in range(len(const.HIDDEN_STATE_COLS)):
        fig.update_yaxes(title_text=f"Activation l_{l}", row=l+1, col=1)

fig.update_layout(height=1000, width=1200, title_text="Latent neuron activations vs. hidden states", showlegend=False)
fig.show()